Author: Ashitabh Misra  
Course: Lightweight Cryptography Term Paper Codebook

# IMPORTS

In [17]:
import textwrap 
import numpy as np
import random

# SBOX's

In [15]:
sbox_0 = [0,6,14,1,15,4,7,13,9,8,12,5,2,10,3,11]
sbox_1 = [0,9,13,2,15,1,11,7,6,4,5,3,8,12,10,14]

# Encryption

## Permutation Layer

In [6]:
def perm_layer(state,r=1):
    new_state = [0 for i in range(64)]
    if r%4 == 1:
        for i in range(64):
            z = int(i/16)
            x = int(i/4) % 4
            y = int(i%4)
            
            x_dash = (x+y)%4
            y_dash = y
            z_dash = z
            
            new_index = z_dash*16 + x_dash*4 +y_dash
            
            new_state[new_index] = state[i]
    if r%4 == 3:
        for i in range(64):
            z = int(i/16)
            x = int(i/4) % 4
            y = int(i%4)

            x_dash = x
            y_dash = y
            z_dash = (y+z)%4

            new_index = z_dash*16 + x_dash*4 +y_dash

            new_state[new_index] = state[i]
    if r%2 ==0:
        new_state = [state[index] for index in range(64)]

    return new_state

## Substitution Layer

In [5]:
def s_layer(state):
    new_state= [0 for index in range(64)]
    for index in range(64):
        if index%2 ==0:
            new_state[index] = sbox_0[state[index]]
        if index%2 ==1:
            new_state[index] = sbox_1[state[index]]
    return new_state

## Linear Layer

In [12]:
mds = np.array([[0,1,0,0],
                [0,0,1,0],
               [0,0,0,1],
               [1,1,0,0]])
mds_2 = np.matmul(mds,mds)




print("MDS")
print(mds)
print("MDS^2")
print(mds_2)

MDS
[[0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 1 0 0]]
MDS^2
[[0 0 1 0]
 [0 0 0 1]
 [1 1 0 0]
 [0 1 1 0]]


In [4]:
def xor(nibs):
    nib_list = [x.astype(np.int) for x in nibs]
        
    t0 = nib_list[0]
    temp = t0
    for t in nib_list[1:]:
        temp = np.bitwise_xor(temp,t)
    return temp


def mc_a(a,b,c,d):
    t1  = np.matmul(mds_2,a)%2
    t2 = np.matmul(mds_2,b)%2
    t3 = np.matmul(mds,b)%2
    t4 = c
    t5 = d
    answer_temp = xor([t1,t2,t3,t4,t5])
    
    return answer_temp%2


def mc_b(a,b,c,d):
    t1  = a
    t2 = np.matmul(mds,b)%2
    t3 = b
    t4 = np.matmul(mds_2,c)%2
    t5 = c
    t6 = np.matmul(mds_2,d)%2
    t7 = np.matmul(mds,d)%2
    t8 = d
    answer_temp = xor([t1,t2,t3,t4,t5,t6,t7,t8])
    
    return answer_temp%2

def mc_c(a,b,c,d):
    t1  = a
    t2 = b
    t3 = np.matmul(mds_2,c)%2
    t4 = np.matmul(mds_2,d)%2
    t5 = np.matmul(mds,d)%2

    answer_temp = xor([t1,t2,t3,t4,t5])
    return answer_temp%2

def mc_d(a,b,c,d):
    t1  = np.matmul(mds_2,a)%2
    t2 = a
    t3 = np.matmul(mds_2,b)%2
    t4 = np.matmul(mds,b)%2
    t5 = b
    t6 = c
    t7 = np.matmul(mds,d)%2
    t8 = d
    answer_temp = xor([t1,t2,t3,t4,t5,t6,t7,t8])
    return answer_temp%2

In [3]:
def linear_layer(state):
    new_state = [0 for i in range(64)]
    i =0
    for column in range(16):
#         print(i)
        a  = column*4
        b  = column*4+1
        c  = column*4+2
        d  = column*4+3
        
        a_bin= get_bin(state[a])
        b_bin= get_bin(state[b])
        c_bin= get_bin(state[c])
        d_bin= get_bin(state[d])
#         print(c_bin)
        
        a_dash = mc_a(a_bin,b_bin,c_bin,d_bin)
        b_dash = mc_b(a_bin,b_bin,c_bin,d_bin)
        c_dash = mc_c(a_bin,b_bin,c_bin,d_bin)
        d_dash = mc_d(a_bin,b_bin,c_bin,d_bin)
        
        a_num = get_num(a_dash)
        b_num = get_num(b_dash)
        c_num = get_num(c_dash)
        d_num = get_num(d_dash)
        
        new_state[column*4] = a_num
        new_state[column*4+1] = b_num
        new_state[column*4+2] = c_num
        new_state[column*4+3] = d_num
        i+=1
  
    return new_state

# Add Round key

In [23]:
def add_round_key(state,key,r):
    init_state = [0 for x in range(64)]
    if r%4 == 3: 
        for i in range(64):
            init_state = state[i] ^ key[i]
        return init_state
    if r%4 == 1:
        for i in range(64):
            init_state[i] = state[i] ^ key[(i+20)%64]
        return init_state
    if r%4 ==0:
        init_state = [state[x] for x in range(64)]
        return init_state

In [21]:
def get_bin(num = None):
    import numpy as np
    num_bin = '{0:04b}'.format(num)
    num_str = list(num_bin)
    
    num_int = [int(x) for x in num_str]
    
    num_np = np.array(num_int,dtype =np.int)
    
    return num_np

def get_num(a):
    a_d = [str(x) for x in list(a)]
    return int("".join(a_d),2)
    

# Branch Number Calculation

In [2]:
def get_bin_hw(state):
#     print(state)
    return_array = np.array([0,0,0,0])
    for x in range(4):
        return_array[x] = int(state[x])
    return return_array


def get_string(state):
    string_return = ''
    for x in state:
        string_return+=str(x)
    return string_return



def column_operation(states):
    a_bin= get_bin_hw(states[0])
    b_bin= get_bin_hw(states[1])
    c_bin= get_bin_hw(states[2])
    d_bin= get_bin_hw(states[3])
    
    a_dash = mc_a(a_bin,b_bin,c_bin,d_bin)
    b_dash = mc_b(a_bin,b_bin,c_bin,d_bin)
    c_dash = mc_c(a_bin,b_bin,c_bin,d_bin)
    d_dash = mc_d(a_bin,b_bin,c_bin,d_bin)
    
    a_return = get_string(a_dash)
    b_return = get_string(b_dash)
    c_return = get_string(c_dash)
    d_return = get_string(d_dash)
    
    return [a_return,b_return,c_return,d_return]
    
    
    
def return_num_active_sbox(states_active):
    num_active_sbox =0
    for state in states_active:
        if '1' in state:
            num_active_sbox+=1
    return num_active_sbox
        

In [14]:
def min_hamming_weight():
    branch_number = 9999
    branch_value_dict = {'5':0,'6':0,'7':0,'8':0}
    for i in range(1,2**16):
        curr_bin = '{0:016b}'.format(i)
        input_nibbles = textwrap.wrap(curr_bin,4)
        input_wt = return_num_active_sbox(input_nibbles)
        output_nibbles = column_operation(input_nibbles)
        output_wt = return_num_active_sbox(output_nibbles)
        if input_wt+output_wt < branch_number:
            branch_number = input_wt + output_wt
        branch_value_dict[str(input_wt+output_wt)]+=1
    print("Branch Number: ",branch_number,"\nTotal value distribution: ",branch_value_dict)
    return branch_number,branch_value_dict
min_hamming_weight()

Branch Number:  5 
Total value distribution:  {'5': 840, '6': 4620, '7': 21000, '8': 39075}


(5, {'5': 840, '6': 4620, '7': 21000, '8': 39075})

# ZERO SUM PROPERTY

Please Intialise the encryption functions before running this

## Number of rounds: 1

In [19]:
def round_fn(state,key,r):
    state = s_layer(state)
    state = perm_layer(state,r)
    state = linear_layer(state)
    state = add_round_key(state,key,r)
    
    return state

In [24]:
key= [3, 6, 12, 1, 11, 9, 1, 10, 3, 7, 3, 11, 0, 6, 3, 0, 6, 14, 9, 11, 13, 2, 7, 10, 10, 2, 6, 14, 2, 9, 13, 11, 11, 2, 11, 5, 5, 10, 14, 5, 5, 11,
       10, 5, 15, 12, 15, 14, 15, 15, 1, 9, 14, 9, 8, 5, 11, 1, 0, 5, 11, 3, 14, 12]


init_list = [x for x in range(0,16)]

pt_list = [[0 for i in range(64)] for j in range(16)]

for i in range(64):
    random.shuffle(init_list)
    for j in range(16):
        pt_list[j][i] = init_list[j]


ct_list = []
for i in range(16):
    ct_list.append(round_fn(pt_list[i],key,0))


for i in range(64):
    curr = 0
    for j in range(16):
        curr ^= ct_list[j][i]
    print(curr, end=' ')

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

## Number of rounds: 2

In [25]:
init_list = [x for x in range(0,16)]

pt_list = [[0 for i in range(64)] for j in range(1024)]

for i in range(64):
    random.shuffle(init_list)
    for j in range(1024):
        pt_list[j][i] = init_list[j%16]

key= [3, 6, 12, 1, 11, 9, 1, 10, 3, 7, 3, 11, 0, 6, 3, 0, 6, 14, 9, 11, 13, 2, 7,
      10, 10, 2, 6, 14, 2, 9, 13, 11, 11, 2, 11, 5, 5, 10, 14, 5, 5, 11,
       10, 5, 15, 12, 15, 14, 15, 15, 1, 9, 14, 9, 8, 5, 11, 1, 0, 5, 11, 3, 14, 12]
key2= [3, 1, 12, 14, 1, 8, 13, 1, 9, 9, 14, 3, 5, 5, 13, 11, 5, 13, 10, 3, 4,
       12, 9, 14, 14, 7, 7, 0, 10, 10, 3, 2, 8, 8, 15, 9, 12, 
       7, 11, 13, 15, 8, 8, 13, 14, 14, 14, 11, 6, 12, 4, 13, 9, 2, 13, 12, 11, 6, 11, 14, 15, 12, 8, 7]

ct_list = []
for i in range(1024):
    state = round_fn(round_fn(pt_list[i],key,0),key2,1)
    ct_list.append(state)



for i in range(64):
    curr = 0
    for j in range(1024):
        curr ^= ct_list[j][i]
    print(curr, end=' ')


0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

# MILP Generation

The code is given in the folder as well. Can use that to generate .lp files and run with gurobi.

## MILP Type 1: CUBE version

In [31]:




BN = 5
def objective_function(r  = 2):
    number_active_sboxs = r*64
    print("Minimize")
    for i in range(number_active_sboxs):
        if i == number_active_sboxs-1:
            print(f"x_{i}",end=" ")
        else:
            print(f"x_{i} +", end = " ")


def linear_layer(num_rounds =2):
    # Intial State
    init_state  = [f'x_{j}' for j in range(64) ]
    for i in range(num_rounds):

        #Subs Layer -- No Change
        #print(f"ROUND NUMBER {i}")
        # Shift Row
        sr_state = perm_layer(init_state,r =i)
        # Linear Layer
        curr_ll = [f"x_{(i+1)*64 + j}" for j in range(64)]
#         print("SR_STATE,", sr_state)
#         print("LL_LAYER,", curr_ll)
        init_state = curr_ll
        for column in range(16):
            a = column*4
            b = column*4+1
            c = column*4+2
            d = column*4+3
            col_index = [a,b,c,d]
            for elem in range(4):
                print(f"{sr_state[col_index[elem]]} ",end =" ")
                print(" + ",end=" ")
                print(f"{curr_ll[col_index[elem]]} ",end =" ")
                if elem != 3:
                    print(" + ",end = " ")
            # Dummy Variables


            print(f"- {BN} d{i*16+column} >= 0")
            for elem in range(4):
                print(f" d{i*16+column}  - {sr_state[col_index[elem]]} >= 0")
                print(f" d{i*16+column}  - {curr_ll[col_index[elem]]} >= 0")

    print("\n\n")


def setting_type(num_rounds =1):
    print("\n\n")
    print("Binary")
    num_active_sboxs = (num_rounds+1)*64
    for i in range(num_active_sboxs):
        print(f"x_{i}")
    for column in range((num_rounds)*16):
        print(f"d{column}")
    print("End")

def perm_layer(state,r=1):
    new_state = [0 for i in range(64)]
    if r%4 == 1:
        for i in range(64):
            z = int(i/16)
            x = int(i/4) % 4
            y = int(i%4)

            x_dash = (x+y)%4
            y_dash = y
            z_dash = z

            new_index = z_dash*16 + x_dash*4 +y_dash

            new_state[new_index] = state[i]
    if r%4 == 3:
        for i in range(64):
            z = int(i/16)
            x = int(i/4) % 4
            y = int(i%4)

            x_dash = x
            y_dash = y
            z_dash = (y+z)%4

            new_index = z_dash*16 + x_dash*4 +y_dash

            new_state[new_index] = state[i]
    if r%2 ==0:
        new_state = [state[index] for index in range(64)]

    return new_state

def atleast_one_active(r):
    number_active_sboxs = r*64
    for i in range(number_active_sboxs):
        if i == number_active_sboxs-1:
            print(f"x_{i}",end=" ")
        else:
            print(f"x_{i} +", end = " ")
    print(" >= 1 ")

ROUNDS =2
objective_function(ROUNDS)
print("\n\nSubject To")
linear_layer(ROUNDS)
atleast_one_active(ROUNDS)
setting_type(ROUNDS)





Minimize
x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9 + x_10 + x_11 + x_12 + x_13 + x_14 + x_15 + x_16 + x_17 + x_18 + x_19 + x_20 + x_21 + x_22 + x_23 + x_24 + x_25 + x_26 + x_27 + x_28 + x_29 + x_30 + x_31 + x_32 + x_33 + x_34 + x_35 + x_36 + x_37 + x_38 + x_39 + x_40 + x_41 + x_42 + x_43 + x_44 + x_45 + x_46 + x_47 + x_48 + x_49 + x_50 + x_51 + x_52 + x_53 + x_54 + x_55 + x_56 + x_57 + x_58 + x_59 + x_60 + x_61 + x_62 + x_63 + x_64 + x_65 + x_66 + x_67 + x_68 + x_69 + x_70 + x_71 + x_72 + x_73 + x_74 + x_75 + x_76 + x_77 + x_78 + x_79 + x_80 + x_81 + x_82 + x_83 + x_84 + x_85 + x_86 + x_87 + x_88 + x_89 + x_90 + x_91 + x_92 + x_93 + x_94 + x_95 + x_96 + x_97 + x_98 + x_99 + x_100 + x_101 + x_102 + x_103 + x_104 + x_105 + x_106 + x_107 + x_108 + x_109 + x_110 + x_111 + x_112 + x_113 + x_114 + x_115 + x_116 + x_117 + x_118 + x_119 + x_120 + x_121 + x_122 + x_123 + x_124 + x_125 + x_126 + x_127 

Subject To
x_0   +  x_64   +  x_1   +  x_65   +  x_2   +  x_66   +  x_3   +  

## MILP Type 2: SQUARE version

In [32]:
def objective_function(r  = 1):
    number_active_sboxs = r*16
    print("Minimize")
    for i in range(number_active_sboxs):
        if i == number_active_sboxs-1:
            print(f"x_{i}",end=" ")
        else:
            print(f"x_{i} +", end = " ")



def perm_layer_transpose(state):
    new_state = [0 for x in range(16)]
    
    for i in range(16):
        x = int(i/4)
        y = i%4
        
        y_dash = x
        x_dash = y
        
        new_index = x_dash*4 + y_dash
        new_state[new_index] = state[i]
    return new_state

def linear_layer_milp(num_rounds =2):
# Intial State
    init_state  = [f'x_{j}' for j in range(64) ]
    for i in range(num_rounds):
        
        #Subs Layer -- No Change
        #print(f"ROUND NUMBER {i}")
        # Shift Row
       
        # Linear Layer
        curr_ll = [f"x_{(i+1)*16 + j}" for j in range(16)]
    #         print("SR_STATE,", sr_state)
    #         print("LL_LAYER,", curr_ll)
    #         if (i-1)%2==0
    #             init_state = curr_ll
        if i>=1:
            init_state = sr_state
        for column in range(4):
            a = column*4
            b = column*4+1
            c = column*4+2
            d = column*4+3
            col_index = [a,b,c,d]
            for elem in range(4):
                print(f"{init_state[col_index[elem]]} ",end =" ")
                print(" + ",end=" ")
                print(f"{curr_ll[col_index[elem]]} ",end =" ")
                if elem != 3:
                    print(" + ",end = " ")
            # Dummy Variables


            print(f"- {BN} d{i*4+column} >= 0")
            for elem in range(4):
                print(f" d{i*4+column}  - {init_state[col_index[elem]]} >= 0")
                print(f" d{i*4+column}  - {curr_ll[col_index[elem]]} >= 0")
        sr_state = perm_layer_transpose(curr_ll)


print("\n\n")                 


BN = 5

def setting_type(num_rounds =2):
    print("\n\n")
    print("Binary")
    num_active_sboxs = (num_rounds+1)*16
    for i in range(num_active_sboxs):
        print(f"x_{i}")
    for column in range((num_rounds)*4):
        print(f"d{column}")
    print("End")


def atleast_one_active(r):
    number_active_sboxs = r*16
    for i in range(number_active_sboxs):
        if i == number_active_sboxs-1:
            print(f"x_{i}",end=" ")
        else:
            print(f"x_{i} +", end = " ")
    print(" >= 1 ")


BN = 5

if __name__ =="__main__":

    ROUNDS =12

    objective_function(ROUNDS)
    print("\n\nSubject To")
    linear_layer_milp(ROUNDS)
    atleast_one_active(ROUNDS)
    setting_type(ROUNDS)





Minimize
x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9 + x_10 + x_11 + x_12 + x_13 + x_14 + x_15 + x_16 + x_17 + x_18 + x_19 + x_20 + x_21 + x_22 + x_23 + x_24 + x_25 + x_26 + x_27 + x_28 + x_29 + x_30 + x_31 + x_32 + x_33 + x_34 + x_35 + x_36 + x_37 + x_38 + x_39 + x_40 + x_41 + x_42 + x_43 + x_44 + x_45 + x_46 + x_47 + x_48 + x_49 + x_50 + x_51 + x_52 + x_53 + x_54 + x_55 + x_56 + x_57 + x_58 + x_59 + x_60 + x_61 + x_62 + x_63 + x_64 + x_65 + x_66 + x_67 + x_68 + x_69 + x_70 + x_71 + x_72 + x_73 + x_74 + x_75 + x_76 + x_77 + x_78 + x_79 + x_80 + x_81 + x_82 + x_83 + x_84 + x_85 + x_86 + x_87 + x_88 + x_89 + x_90 + x_91 + x_92 + x_93 + x_94 + x_95 + x_96 + x_97 + x_98 + x_99 + x_100 + x_101 + x_102 + x_103 + x_104 + x_105 + x_106 + x_107 + x_108 + x_109 + x_110 + x_111 + x_112 + x_113 + x_114 + x_115 + x_116 + x_117 + x_118 + x_119 + x_120 + x_121 + x_122 + x_123 + x_124 + x_125 + x_126 + x_127 + x_128 + x_129 + x_130 + x_131 + x_132 + x_133 + x_134 + x_135 + x_136 + x_

# SBOX Properties 

Please run the file <file_name> given in the folder.

# BCT

In [26]:

sbox_0 = [0,6,14,1,15,4,7,13,9,8,12,5,2,10,3,11]


inv_sbox_0 = [0 for i in range(16)]
for j in range(16):
    inv_sbox_0[sbox_0[j]] = j
print("SBOX 0:", sbox_0)
print("INVERSE SBOX 0: ", inv_sbox_0)


sbox_1 = [0,9,13,2,15,1,11,7,6,4,5,3,8,12,10,14]

inv_sbox_1 = [0 for i in range(16)]
for j in range(16):
    inv_sbox_1[sbox_1[j]] = j
print("SBOX 1:", sbox_1)
print("INVERSE SBOX 1: ", inv_sbox_1)

SBOX 0: [0, 6, 14, 1, 15, 4, 7, 13, 9, 8, 12, 5, 2, 10, 3, 11]
INVERSE SBOX 0:  [0, 3, 12, 14, 5, 11, 1, 6, 9, 8, 13, 15, 10, 7, 2, 4]
SBOX 1: [0, 9, 13, 2, 15, 1, 11, 7, 6, 4, 5, 3, 8, 12, 10, 14]
INVERSE SBOX 1:  [0, 5, 3, 11, 9, 10, 8, 7, 12, 1, 14, 6, 13, 2, 15, 4]


##  BCT SBOX 0

In [27]:
sbox = sbox_0
inv_sbox = inv_sbox_0

bct = [[0 for i in range(16)] for i in range(16)]



def get_val_bct(x,d0,d1):
    return inv_sbox[sbox[x] ^ d0] ^ inv_sbox[sbox[x ^ d1] ^ d0]



def do_assignment(d1,d0):
    bct[d1][d0]+=1

    
    
    
[[[ do_assignment(d1,d0)  for x in range(16) if get_val_bct(x,d0,d1) == d1] for d0 in range(16)] for d1 in range(16)]

for i in bct:
    print(i)



[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]
[16, 6, 0, 0, 0, 0, 2, 0, 4, 6, 2, 2, 0, 0, 0, 2]
[16, 4, 0, 0, 0, 2, 0, 2, 6, 6, 0, 0, 0, 2, 2, 0]
[16, 6, 2, 2, 2, 0, 0, 0, 6, 4, 0, 0, 2, 0, 0, 0]
[16, 0, 4, 4, 0, 0, 0, 0, 0, 2, 0, 2, 10, 0, 6, 4]
[16, 0, 4, 4, 10, 0, 4, 6, 0, 2, 2, 0, 0, 0, 0, 0]
[16, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2]
[16, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0]
[16, 0, 10, 0, 4, 0, 6, 0, 0, 2, 0, 0, 4, 2, 4, 0]
[16, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2]
[16, 0, 0, 10, 4, 2, 0, 4, 0, 2, 0, 0, 4, 0, 0, 6]
[16, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2]
[16, 0, 6, 0, 0, 0, 2, 0, 2, 0, 2, 2, 4, 2, 4, 0]
[16, 2, 4, 0, 6, 2, 4, 2, 0, 0, 2, 0, 0, 2, 0, 0]
[16, 2, 0, 4, 0, 2, 0, 0, 0, 0, 0, 2, 6, 2, 2, 4]
[16, 0, 0, 6, 4, 2, 0, 4, 2, 0, 2, 2, 0, 0, 0, 2]


## BCT SBOX 1

In [28]:
sbox = sbox_1
inv_sbox = inv_sbox_1

bct = [[0 for i in range(16)] for i in range(16)]



def get_val_bct(x,d0,d1):
    return inv_sbox[sbox[x] ^ d0] ^ inv_sbox[sbox[x ^ d1] ^ d0]



def do_assignment(d1,d0):
    bct[d1][d0]+=1

    
    
    
[[[ do_assignment(d1,d0)  for x in range(16) if get_val_bct(x,d0,d1) == d1] for d0 in range(16)] for d1 in range(16)]

for i in bct:
    print(i)




[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]
[16, 0, 6, 0, 4, 0, 6, 0, 0, 2, 0, 0, 2, 0, 2, 2]
[16, 0, 4, 2, 6, 0, 6, 2, 0, 0, 0, 2, 0, 2, 0, 0]
[16, 2, 6, 0, 6, 2, 4, 0, 2, 0, 2, 0, 0, 0, 0, 0]
[16, 0, 0, 0, 0, 10, 2, 0, 4, 0, 4, 0, 0, 6, 2, 4]
[16, 10, 0, 0, 0, 0, 2, 0, 4, 4, 4, 6, 2, 0, 0, 0]
[16, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2]
[16, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0]
[16, 4, 0, 0, 0, 4, 2, 2, 10, 6, 0, 0, 0, 4, 0, 0]
[16, 0, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2]
[16, 4, 0, 2, 0, 4, 2, 0, 0, 0, 10, 4, 0, 0, 0, 6]
[16, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2]
[16, 0, 0, 0, 2, 4, 0, 2, 6, 2, 0, 0, 2, 4, 2, 0]
[16, 6, 2, 2, 0, 0, 0, 2, 4, 4, 0, 2, 2, 0, 0, 0]
[16, 0, 2, 2, 0, 6, 0, 2, 0, 0, 4, 0, 0, 2, 2, 4]
[16, 4, 0, 2, 2, 0, 0, 0, 0, 0, 6, 4, 2, 0, 2, 2]


# BDT

## SBOX 0

In [29]:
bdt = {}

for j in range(16):
    bdt[j] = [[0 for i in range(16)] for i in range(16)]
def get_c_d0(x,d0,invd0):
    return inv_sbox_0[sbox_0[x] ^ invd0] ^ inv_sbox_0[sbox_0[x ^ d0] ^ invd0]

def get_c_d1(x,d0):
    return  sbox_0[x] ^ sbox_0[x ^ d0]

def do_assignment(d1,d0,invd0):
    bdt[d1][d0][invd0] += 1


[[[[do_assignment(d1,d0,invd0) for x in range(16) if get_c_d0(x,d0,invd0) == d0 and get_c_d1(x,d0) == d1] for invd0 in range(16)] for d0 in range(16)] for d1 in range(16)]



for i in bdt:
    print("*"*25, f" d1 = {i}", "*"*25)
    for i in bdt[i]:
        print(i)
    print("\n")



*************************  d1 = 0 *************************
[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


*************************  d1 = 1 *************************
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 2, 0, 0, 0, 0, 0, 0, 0, 0

## SBOX 1 

In [30]:
bdt = {}

for j in range(16):
    bdt[j] = [[0 for i in range(16)] for i in range(16)]
def get_c_d0(x,d0,invd0):
    return inv_sbox_1[sbox_1[x] ^ invd0] ^ inv_sbox_1[sbox_1[x ^ d0] ^ invd0]

def get_c_d1(x,d0):
    return  sbox_1[x] ^ sbox_1[x ^ d0]

def do_assignment(d1,d0,invd0):
    bdt[d1][d0][invd0] += 1



[[[[do_assignment(d1,d0,invd0) for x in range(16) if get_c_d0(x,d0,invd0) == d0 and get_c_d1(x,d0) == d1] for invd0 in range(16)] for d0 in range(16)] for d1 in range(16)]





for i in bdt:
    print("*"*25, f" d1 = {i}", "*"*25)
    for i in bdt[i]:
        print(i)
    print("\n")



*************************  d1 = 0 *************************
[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


*************************  d1 = 1 *************************
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0